# 4.1 - Procesos asíncronos


![async](images/async.png)




**[Documentación](https://docs.python.org/3/library/asyncio.html)**


**asyncio** es una biblioteca para escribir código [concurrente](https://es.wikipedia.org/wiki/Concurrencia_(inform%C3%A1tica)) utilizando la sintaxis async/await. Se utiliza como base en múltiples frameworks asíncronos de Python y provee un alto rendimiento en redes y servidores web, bibliotecas de conexión de base de datos, colas de tareas distribuidas, etc.

Suele encajar perfectamente para operaciones con límite de E/S y código de red estructurado de alto nivel. Además provee un conjunto de APIs de alto nivel para:

+ ejecutar corutinas de Python de manera concurrente y tener control total sobre su ejecución

+ realizar redes E/S y comunicación entre procesos(IPC)

+ controlar subprocesos

+ distribuir tareas a través de colas

+ sincronizar código concurrente

Adicionalmente, existen APIs de bajo nivel para desarrolladores de bibliotecas y frameworks para:

+ crear y administrar bucles de eventos, los cuales proveen APIs asíncronas para redes, ejecutando subprocesos, gestionando señales del sistema operativo, etc..

+ implementar protocolos eficientes utilizando transportes

+ bibliotecas puente basadas en retrollamadas y código con sintaxis async/wait

$$$$

Nosotros nos enfocaremos en el uso de bucles de eventos para la extracción de datos de la web.


### Hola Mundo

In [1]:
import asyncio

# funcion asincrona

async def saludar():
    
    print('Hola...')
    await asyncio.sleep(3)
    print('..Mundo!!!')
    print('Pero que pashaaa')
    
    
await saludar()          # en jupyter
#asyncio.run(saludar())  # en python .py

Hola...
..Mundo!!!
Pero que pashaaa


### Ejemplo response

Comprobando la respuesta de tres urls.

In [2]:
import requests as req

In [3]:
url='https://s3-eu-west-1.amazonaws.com/'

req.get(url)

<Response [200]>

In [4]:
urls=[
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/breakfast.jpg',
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/forbidden',
    'https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/data-static/documents/the-html5-breakfast-site.html'
]

In [5]:
for url in urls:
    print(req.get(url))

<Response [200]>
<Response [403]>
<Response [200]>


In [6]:
async def comprobar():
    
    bucle=asyncio.get_event_loop()   # bucle asincrono
    
    futuros=[bucle.run_in_executor(None, req.get, url) for url in urls]
    
    for res in await asyncio.gather(*futuros):   # aqui se los pido
        print(res.status_code)
    
await comprobar()

200
403
200


### Ejemplo ESPN

Volvamos al ejemplo de scrapeo de la págine de ESPN. Vamos a realizar múltiples requests para obtener los datos de todos los equipos.


https://www.espn.com/soccer/competitions

In [7]:
from selenium import webdriver

import time

import pandas as pd

from webdriver_manager.chrome import ChromeDriverManager

PATH=ChromeDriverManager().install()

import warnings
warnings.filterwarnings('ignore')



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [/Users/iudh/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.53/chromedriver] found in cache


In [8]:
url='https://www.espn.com/soccer/competitions'

In [ ]:
driver=webdriver.Chrome(PATH)

driver.get(url)

# cookies
aceptar=driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
aceptar.click()

time.sleep(2)

# selecciona equipos laliga
equipos=driver.find_element_by_css_selector('#fittPageContainer > div:nth-child(4) > div > div.layout__column.layout__column--1 > div > div:nth-child(3) > div:nth-child(2) > div > div:nth-child(4) > div > section > div > div > span:nth-child(2) > a')
equipos.click()

time.sleep(2)


In [17]:
equipos_stats_urls=driver.find_elements_by_css_selector('a.AnchorLink')

equipos_stats_urls=[e.get_attribute('href') for e in equipos_stats_urls if '/team/stats/' in e.get_attribute('href')]

driver.quit()

equipos_stats_urls

['https://www.espn.com/soccer/team/stats/_/id/96/alaves',
 'https://www.espn.com/soccer/team/stats/_/id/93/athletic-club',
 'https://www.espn.com/soccer/team/stats/_/id/1068/atletico-madrid',
 'https://www.espn.com/soccer/team/stats/_/id/83/barcelona',
 'https://www.espn.com/soccer/team/stats/_/id/85/celta-vigo',
 'https://www.espn.com/soccer/team/stats/_/id/3842/cadiz',
 'https://www.espn.com/soccer/team/stats/_/id/3751/elche',
 'https://www.espn.com/soccer/team/stats/_/id/88/espanyol',
 'https://www.espn.com/soccer/team/stats/_/id/2922/getafe',
 'https://www.espn.com/soccer/team/stats/_/id/3747/granada',
 'https://www.espn.com/soccer/team/stats/_/id/1538/levante',
 'https://www.espn.com/soccer/team/stats/_/id/84/mallorca',
 'https://www.espn.com/soccer/team/stats/_/id/97/osasuna',
 'https://www.espn.com/soccer/team/stats/_/id/101/rayo-vallecano',
 'https://www.espn.com/soccer/team/stats/_/id/244/real-betis',
 'https://www.espn.com/soccer/team/stats/_/id/86/real-madrid',
 'https://www

**Extracción asincrónica**

In [18]:
def asincrono(funcion):
    def eventos(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, funcion, *args, **kwargs)
    return eventos

In [19]:
DATOS=[]
CABECERAS=[]

In [20]:
@asincrono
def extraer(url):
    
    global DATOS, CABECERAS
    
    # inicia el driver
    driver=webdriver.Chrome(PATH)
    driver.get(url)

    time.sleep(2)

    # acepta cookies
    aceptar=driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
    aceptar.click()

    time.sleep(2)


    disciplina=driver.find_element_by_xpath('//*[@id="fittPageContainer"]/div[2]/div[5]/div/div[1]/section/div/div[2]/nav/ul/li[2]/a')
    disciplina.click()

    time.sleep(2)

    tabla=driver.find_element_by_tag_name('tbody')

    filas=tabla.find_elements_by_tag_name('tr')


    datos=[[e.text for e in f.find_elements_by_tag_name('td')]+[url.split('/')[-1]] for f in filas]

    cabeceras=driver.find_element_by_tag_name('thead')

    cabeceras=[c.text for c in cabeceras.find_elements_by_tag_name('th')]+['TEAM']

    DATOS+=datos

    CABECERAS=cabeceras

In [21]:
for url in equipos_stats_urls[:3]:
    res=extraer(url)
    display(res)

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/iudh/miniforge3/envs/clase/lib/python3.9/asyncio/futures.py:384]>

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/iudh/miniforge3/envs/clase/lib/python3.9/asyncio/futures.py:384]>

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/iudh/miniforge3/envs/clase/lib/python3.9/asyncio/futures.py:384]>

Future exception was never retrieved
future: <Future finished exception=WebDriverException('unknown error: unexpected command response\n  (Session info: chrome=103.0.5060.53)', None, ['0   chromedriver                        0x0000000102b9dd14 chromedriver + 3792148', '1   chromedriver                        0x0000000102b34828 chromedriver + 3360808', '2   chromedriver                        0x00000001028350b8 chromedriver + 217272', '3   chromedriver                        0x0000000102824124 chromedriver + 147748', '4   chromedriver                        0x00000001028238f4 chromedriver + 145652', '5   chromedriver                        0x0000000102822f50 chromedriver + 143184', '6   chromedriver                        0x00000001028227f8 chromedriver + 141304', '7   chromedriver                        0x00000001028226e8 chromedriver + 141032', '8   chromedriver                        0x000000010288d348 chromedriver + 578376', '9   chromedriver                        0x000000010285a31

In [ ]:
#driver.quit()    # cuidao, necesitan cumplirse todas las promesas

In [22]:
df=pd.DataFrame(DATOS, columns=CABECERAS)

df.head()

,RK,NAME,P,YC,RC,PTS,TEAM
0,1,Geoffrey Kondogbia,28,11,2,17,atletico-madrid
1,2,Stefan Savic,28,12,0,12,atletico-madrid
2,3,Mario Hermoso,26,6,1,9,atletico-madrid
3,4,Felipe,26,5,1,8,atletico-madrid
4,,José Giménez,24,8,0,8,atletico-madrid


In [23]:
df.TEAM.unique()

array(['atletico-madrid', 'alaves'], dtype=object)

In [24]:
df.shape

(46, 7)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   RK      46 non-null     object
 1   NAME    46 non-null     object
 2   P       46 non-null     object
 3   YC      46 non-null     object
 4   RC      46 non-null     object
 5   PTS     46 non-null     object
 6   TEAM    46 non-null     object
dtypes: object(7)
memory usage: 2.6+ KB
